In [1]:
import copy

import numpy as np
import torch

from loader.DataLoader import read_dataframe
from loader.DataTransformer import lag_list, transform_matrix
from model.CnnLstmModel import CnnLstmModel
from model.FcLstmModel import FcLstmModel
from model.LstmModel import LstmModel

In [2]:
import matplotlib.pyplot as plt

In [3]:
# data parameter
LAG = 15

In [4]:
# prepare data
sequence = read_dataframe('all').to_numpy()
sequence = sequence[:, 1:]
# sequence = transform_matrix(sequence, 'DIFF')
sequence = transform_matrix(sequence, 'NORM')
# sequence[np.isnan(sequence)] = 0 # fill na - there is a column which are all 0

shifted_sequence = lag_list(sequence, LAG + 1)  # shift into delayed sequences

x_train = shifted_sequence[:, :-1, 1:]  # for each delayed sequence, take all elements except last element
y_train = shifted_sequence[:, -1, -1]  # for each delayed sequence, only take the last element
y_train = y_train.reshape(-1, 1)

x_train = torch.from_numpy(x_train.astype('float64')).type(torch.Tensor)  # convert to tensor
y_train = torch.from_numpy(y_train.astype('float64')).type(torch.Tensor)  # convert to tensor

wave_1_x, wave_1_y = x_train[52 -LAG:103 -LAG], y_train[52 -LAG:103 -LAG]
wave_2_x, wave_2_y = x_train[160 -LAG:280 -LAG], y_train[160 -LAG:280 -LAG]
wave_3_x, wave_3_y = x_train[280 -LAG:505 -LAG], y_train[280 -LAG:505 -LAG]
wave_4_x, wave_4_y = x_train[716 -LAG:738 -LAG], y_train[716 -LAG:738 -LAG]

wave_x, wave_y = wave_4_x, wave_4_y
TRAIN_RANGE = int(np.floor(wave_y.shape[0] * 4 / 5))

### Main Scenario

In [5]:
def train_test_vis(input_x, input_y, train_range, model, learning_rate=1e-5, num_epochs=3_000):
    # train
    loss_fn = torch.nn.MSELoss()
    # loss_fn = torch.nn.L1Loss()
    # loss_fn = torch.nn.SmoothL1Loss()
    optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
    min_loss = np.inf
    best_model_state = None
    model.train()
    y_var = np.var(input_y.numpy().reshape(-1)[:train_range])
    for epoch in range(1, num_epochs + 1):
        y_pred = model(input_x[:train_range])
        loss = loss_fn(y_pred, input_y[:train_range])
        if epoch % 100 == 0:
            print("Epoch: %d | MSE: %.2E | RRSE: %.2E" % (epoch, loss.item(), np.sqrt(loss.item() / y_var)))
        if min_loss > loss.item():
            best_model_state = copy.deepcopy(model.state_dict())
            min_loss = loss.item()
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    model.eval()
    model.load_state_dict(best_model_state)

    x_test = input_x[train_range].reshape(1, LAG, input_x.shape[2])
    y_pred = []
    prediction_range = input_x.shape[0] - train_range
    # prediction_range = 7
    for i in range(prediction_range):
        _ = model(x_test)
        x_test = input_x[train_range + i].reshape(1, LAG, input_x.shape[2])
        x_test[0][-1][0] = torch.tensor(_.item())
        _2 = _.detach().numpy()  # revert from tensor
        _2 = _2.reshape(-1)  # reshape back to normal list
        y_pred.append(_2)

    y_pred = np.array(y_pred).reshape(-1)  # reshape back to normal list
    print("sample prediction:  ", y_pred)

    y_train_sample = input_y[train_range:].detach().numpy().reshape(-1)[:prediction_range]
    print("sample true result: ", y_train_sample)

    mse = sum((y_train_sample - y_pred) ** 2) / len(y_pred)
    rmse = mse / np.var(y_train_sample)
    print("TEST | MSE: %.2E | RRSE: %.2E" % (mse, rmse))
    return mse, rmse, y_pred


## Train and Test

In [6]:
input_dim = x_train.shape[-1]

In [7]:
# model_fclstm = FcLstmModel(input_dim, 128, 3, 1, LAG, 0.1, 0)
# mse_fclstm, rmse_fclstm, y_pred_fclstm =  train_test_vis(wave_x, wave_y, TRAIN_RANGE, model_fclstm, learning_rate=1e-5)

In [8]:
# model_cnnlstm = CnnLstmModel(input_dim, LAG + 1)
# mse_cnnlstm, rmse_cnnlstm, y_pred_cnnlstm =  train_test_vis(wave_x, wave_y, TRAIN_RANGE, model_cnnlstm)

In [9]:
# model_lstm = LstmModel(input_dim, 128, 3, 1)
# mse_lstm, rmse_lstm, y_pred_lstm = train_test_vis(wave_x, wave_y, TRAIN_RANGE, model_lstm, learning_rate=1e-6)

### Plot Result

In [10]:
# y_given = wave_y.detach().numpy().reshape(-1)
# plt.plot(range(len(y_given)), y_given, color='black')
# plt.plot(range(TRAIN_RANGE, len(y_given)), y_pred_lstm, 'b--', label='lstm (mse={mse:.2e})'.format(mse=mse_lstm))
# plt.plot(range(TRAIN_RANGE, len(y_given)), y_pred_cnnlstm, 'g--', label='cnnlstm (mse={mse:.2e})'.format(mse=mse_cnnlstm))
# plt.plot(range(TRAIN_RANGE, len(y_given)), y_pred_fclstm, 'y--', label='fclstm (mse={mse:.2e})'.format(mse=mse_fclstm))
# plt.axvline(x=TRAIN_RANGE, linestyle='--', color='red')
# plt.ylabel("count")
# plt.xlabel("day")
# plt.legend()
# plt.show()

In [11]:
def denorm(seq, seq_min, seq_max):
    return seq * (seq_max - seq_min) + seq_min

In [12]:
# overall_min, overall_max = 0, 56827

# y_given_ = denorm(y_given, overall_min, overall_max)
# y_pred_lstm_ = denorm(y_pred_lstm, overall_min, overall_max)
# y_pred_cnnlstm_ = denorm(y_pred_cnnlstm, overall_min, overall_max)
# y_pred_fclstm_ = denorm(y_pred_fclstm, overall_min, overall_max)

In [13]:
# plt.plot(range(len(y_given_)), y_given_, color='black')
# plt.plot(range(TRAIN_RANGE, len(y_given_)), y_pred_lstm_, 'b--', label='lstm (rmse={rmse:.2e})'.format(rmse=rmse_lstm))
# plt.plot(range(TRAIN_RANGE, len(y_given_)), y_pred_cnnlstm_, 'g--', label='cnnlstm (rmse={rmse:.2e})'.format(rmse=rmse_cnnlstm))
# plt.plot(range(TRAIN_RANGE, len(y_given_)), y_pred_fclstm_, 'y--', label='fclstm (rmse={rmse:.2e})'.format(rmse=rmse_fclstm))
# plt.axvline(x=TRAIN_RANGE, linestyle='--', color='red')
# plt.ylabel("count")
# plt.xlabel("day")
# plt.legend()
# plt.show()